Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved. SPDX-License-Identifier: Apache-2.0

# Modeling molecular SMILES data with Amazon Neptune and RDKit

This notebook walks through the process of modeling chemical structures using Amazon Neptune. It includes transforming a chemical compound represented as a SMILES string into graph data, with nodes representing individual atoms and edges representing the bonds between atoms.This notebook then closes with a visualization and exploration of the molecule caffeine.

   - [Background](#Background)

   - [Solution Overview](#Solution-Overview)

   - [Package Setup](#Package-Setup)

   - [Graph Data Model](#Graph-Data-Model)

   - [RDKit Processing](#RDKit-Processing)

   - [Amazon Neptune Data Upload](#Data-Upload)

   - [Basic Visualization & Queries](#Basic-Visualization-&-Queries)

   - [Clean Up](#Clean-Up)

## Background

   Modeling chemical structures can be a complex and tedious process, even with the help of modern programs and technology. The ability to explore chemical structures at the most fundamental level of atoms and the bonds that connect them is an essential process in [drug discovery, pharmaceutical research](https://pubs.acs.org/doi/abs/10.1021/acs.jcim.0c00947), and [chemical engineering](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8130509/). By infusing chemical research with technology, researchers can expedite outcome timelines, identify hidden relationships, and overall simplify a traditionally complex process. 

## Solution Overview

In order to integrate technology into the analysis of chemical structures, molecules themselves must first be represented in a machine-readable format, such as [SMILES (simplified molecular-input line-entry system)](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system). **SMILES** format strings are the industry standard in representing molecular structures. The SMILES format enables the relationships between atoms in a molecular structure to be conveyed as a machine processable string. The SMILES format is not all encompassing, leaving out details such as certain polarities and bond properties. However, SMILES does enable powerful analysis at scale of different structures. 

Using Amazon Neptune and the open-source chemical-Informaics software package [RDKit](https://www.rdkit.org/), SMILES format data can be ingested, processed, and converted into nodes and edges in a property graph. Modeling molecular structures in a graph database allows for powerful custom visualization and manipulation at the scale demanded by pharmaceutical applications. Utilizing a graph database such as Neptune allows users to compare millions of molecules with millions of associated interactions. Additionally, the fully managed and serverless infrastructure allows experts with backgrounds in biology and chemistry to focus primarily on the research outcomes of their graph data, avoiding the undifferentiated heavy lifting of managing a complex graph database infrastructure.

This walkthrough follows the process of converting a singular SMILES string, **caffeine**, to graph data in Neptune. However, the process will work for any SMILES format string you would like to use. We’re sourcing the string for caffeine from the [National Library of Medicine](https://pubchem.ncbi.nlm.nih.gov/compound/2519#section=InChI), which maintains a public dataset of many chemical structures [**CN1C=NC2=C1C(=O)N(C(=O)N2C)C**].


We also use the open-source cheminformatics package [RDKit](https://github.com/rdkit/rdkit), Python-based data science package [Pandas](https://pandas.pydata.org/), and [AWS SDK for pandas (awswrangler)](https://aws-sdk-pandas.readthedocs.io/en/stable/). RDKit has a strong community and a great number of chem-informatics utilities; we’ll only be exploring a small portion for this post. Pandas is an open-source Python-based data science toolkit with large community support. The AWS SDK for pandas provides a large set of tools to help AWS services interact with the pandas library.

## Package Setup

The first step in modeling a chemical structure as graph data is importing the required packages. Here we will be using **RDKit**, **Pandas**, and **awswrangler**

In [ ]:
%pip install rdkit

In [ ]:
%pip install awswrangler

In [ ]:
from rdkit import Chem
import pandas as pd
import awswrangler as wr

## Graph Data Model

There are a few different options for graph query languages and their associated data models when working with Neptune; in this case we’re using **Apache TinkerPop’s Gremlin**. We are opting for Gremlin here due to its intuitive nature and easy to learn syntax. The cell below is defining a dictionary object for both the nodes and edges of our graph. Within each dictionary object is a set of properties we will gather from our caffeine molecule in the next section.

In [ ]:
nodes_dict = {'~id':[],
                '~label':[],
                'idx':[],
                'atomicNumber':[],
                'isAromatic': []
               }
               
edges_dict = {'~id':[],
                '~label':[],
                '~from':[],
                '~to':[],
               }

## RDKit Processing

This section is where the chemical computing magic happens. We use the `rdkit` package installed earlier in the graph notebook to decompose our chemical structure into lists of **nodes (atoms)** and **edges (bonds)**.

First, we want to declare our SMILES string for the caffeine molecule as a variable.

In [ ]:
caffeine_smiles = 'CN1C=NC2=C1C(=O)N(C(=O)N2C)C'

Next, to obtain a molecule-type object from the RDKit package, we need to use the below call to the `Chem` library from RDKit imported earlier.

In [ ]:
mol = Chem.MolFromSmiles(caffeine_smiles)

Let’s see a visual of our work so far, run the following cell to output a 2D picture of our molecule below.

In [ ]:
mol

To recap what we just did, first we declared our SMILES string for caffeine as the variable `caffeine_smiles`. Next, we used the `Chem.MolFromSmiles` function from RDKit to turn the SMILES into a `mol` type object defined by RDKit. Finally, we returned the `mol` type object which resulted in a 2D image of the molecular structure for caffeine that we were working with.

Now we need to iterate through each atom and bond within the `mol` object outputted from RDKit. While iterating through each atom and bond, we use the graph data model we declared earlier, storing properties of each inside the data model. Feel free to dive deeper into the `mol.GetAtoms()` and `mol.GetBonds()` function calls on your own - we are only exploring a small subset of their functionality.

In [ ]:
for atom in mol.GetAtoms():
    nodes_dict['~id'].append('Node-'+ caffeine_smiles + str(atom.GetIdx()))
    nodes_dict['~label'].append(atom.GetSymbol())
    nodes_dict['idx'].append(atom.GetIdx())
    nodes_dict['atomicNumber'].append(atom.GetAtomicNum())
    nodes_dict['isAromatic'].append(atom.GetIsAromatic())

for bond in mol.GetBonds():
    edges_dict['~id'].append('edge-'+ caffeine_smiles + str(bond.GetBeginAtomIdx()) + str(bond.GetEndAtomIdx()))
    edges_dict['~label'].append(str(bond.GetBondType()))
    edges_dict['~from'].append('Node-' + caffeine_smiles + str(bond.GetBeginAtomIdx()))
    edges_dict['~to'].append('Node-' + caffeine_smiles + str(bond.GetEndAtomIdx()))

Several different RDKit functions are in this portion of code, so let’s break it down piece by piece: 

•	For the `~id` field of both nodes and edges, we combine the data type `Node` or `Edge`, the SMILES string, and the unique index for the atom

•	For the `~label` field, we use the chemical symbol for nodes, and the bond type for the edges

•	The fields `~from` and `~to` for the edges (bonds) are constructed by combining the prefix `Node-` with the SMILES string, and the respective beginning and ending atoms that the bond connects

•	The additional fields for the nodes (atoms) in the graph model are the atom’s unique ID within the molecule, its atomic number, and if it is aromatic or not

Note that you can extract several atomic properties for a given SMILES string in RDKit and add them as additional fields for a given atom or bond. We don’t list them all in this post, but you can explore additional fields for both the atoms and bonds.

Finally, we want to use **Pandas** to transform our data-poulated dictionaries into pandas data frames

In [ ]:
nodes_df = pd.DataFrame.from_dict(nodes_dict)
edges_df = pd.DataFrame.from_dict(edges_dict)

let's check the results of our work so far. Running the cells below should return data frames for both the edges and nodes of our caffeine molecule.

In [ ]:
edges_df

In [ ]:
nodes_df

## Amazon Neptune Data Upload

Now that we have successfully decomposed our caffeine SMILES string into individual atoms and bonds, the next step is to load our data into the Neptune database itself. This will be much simpler than loading data from an external source because our data is already inside the graph notebook environment. In order to write our data to the Neptune database, we will be using the **AWS SDK for pandas**, also known as **awswrangler**

First, we need to check our notebook configuration to gather the host endpoint for our cluster. Running the cell below will provide that information, along with other important details about our Neptune database.

In [ ]:
%graph_notebook_config

Find the `host` field from the above output and copy & paste the string into the cell below where it says `'[INSERT YOUR HOST HERE]'`. Also, check the port number above, ensure that the port number above is the same as the second parameter in the cell below. The default port number should be `8182`.

Run cell below once the host & port is properly copied from your graph configuration output above. This cell is simply using a command from the Neptune section of the awswrangler library to establish a connection to your Neptune instance.

In [ ]:
client = wr.neptune.connect("INSERT YOUR HOST HERE", 8182, iam_enabled=False)

The next two cells use the `.to_property_graph` functions within awswrangler to insert both the node & edge data frames we created earlier into our Neptune database. Both cells should return a `"True"` upon success.

In [ ]:
wr.neptune.to_property_graph(client, df=nodes_df)

In [ ]:
wr.neptune.to_property_graph(client, df=edges_df)

After receiving a `"True"` output from both cells, you are finished with processing your SMILES molecule string. Now you can move onto visualizing your molecule as graph data. If you wish to add additional compounds to your graph database, you can return to the start of the [RDKit Processing](#RDKit-Processing) section and simply redo the process with a different SMILES string.

## Basic Visualization & Queries

Now that we have processed and loaded our molecular data, let's visualize the results of our efforts so far. The below cell uses a Gremlin query to traverse outwards from nodes which are labeled as **'C'** (*for the element Carbon*), giving us a picture of the overall structure of our molecule.

In [ ]:
%%gremlin -p v,ine,outv,oute,inv,oute,inv,oute,inv,oute,inv
g.V().has('~label','C').repeat(outE().inV()).emit().times(5).path().by(valueMap(true))

You should receive an interactive view of the 2D image RDKit produced earlier as the output of the above cell. Feel free to explore the structure and compare it to other visualizations you might be able to find of your molecule. Also, now that our molecule is persisted as graph data, you can manipulate, edit, and add additional data to your molecular structure as you see fit.

## Conclusion

In this guide, you ingested and parsed a **SMILES** format molecular data string with **RDKit** and
uploaded the individual atoms and bonds as graph data to **Amazon Neptune**. You can replicate this
process at *scale* to accommodate large datasets containing many SMILES strings. You can test
this yourself by following the steps for any SMILES string of your choice. With the molecular
data broken into individual atoms and bonds in Neptune, you can connect this data to **custom
bioinformatics applications**, **chemical computing systems**, and **research software environments**.
You can take this solution even further by integrating [Amazon Neptune ML](https://aws.amazon.com/blogs/database/how-to-get-started-with-neptune-ml/) to gain the ability to
predict the connections and properties of your molecules.

**See these other resources below to learn more about Amazon Neptune's role in Healthcare & Life Sciences:**

- [Accelerating drug discovery through knowledge graphs](https://aws.amazon.com/blogs/industries/accelerating-drug-discovery-through-knowledge-graph/)

- [Analyze healthcare FHIR data with Amazon Neptune](https://aws.amazon.com/blogs/database/analyze-healthcare-fhir-data-with-amazon-neptune/)

- [Building Amazon Neptune based MedDRA terminology mapping for pharmacovigilance and adverse event reporting](https://aws.amazon.com/blogs/industries/building-amazon-neptune-based-meddra-terminology-mapping-for-pharmacovigilance-and-adverse-event-reporting/)

- [Building and querying the AWS COVID-19 knowledge graph](https://aws.amazon.com/blogs/database/building-and-querying-the-aws-covid-19-knowledge-graph/)

- [General Neptune Developer Resources](https://aws.amazon.com/neptune/developer-resources/)

## Clean Up

*WILL DELETE MOLECULE DATA*

Run the cell below to delete the data in the graph if you want to clean your graph database storage. This iterates through the `nodes_df`, so be sure to adjust accordingly if you have added any of your own additional molecules or edits.

In [ ]:
for i in nodes_df['~id']:
    wr.neptune.execute_gremlin(client, "g.V().has('~id', '"+i+"').drop();")